# 🎨 Fooocus Batch Generator - Automatikus Képgenerálás

Ez a notebook **automatikusan generálja** a képeket TXT fájlokból.

## 📋 Használat

1. **Futtasd az 1. cellát** - Fooocus telepítése
2. **Futtasd a 2. cellát** - TXT fájlok feltöltése
3. **Futtasd a 3. cellát** - Beállítások (opcionális)
4. **Futtasd a 4. cellát** - AUTOMATIKUS GENERÁLÁS
5. **Futtasd az 5. cellát** - Képek letöltése

## 1️⃣ Fooocus Telepítése és Inicializálás

In [ ]:
import os
import json

# Fooocus klónozása
if not os.path.exists("/content/Fooocus"):
    print("📥 Fooocus letöltése...")
    !git clone https://github.com/lllyasviel/Fooocus.git

# Függőségek telepítése
!apt-get -y install -qq aria2
!pip install pygit2==1.15.1 -q

print("✓ Telepítés kész!")

## 2️⃣ TXT Fájlok Feltöltése

Töltsd fel a TXT fájljaidat! **Formátum**:
- Egy sor = egy prompt
- `#`-tel kezdődő sorok = megjegyzések (ignorálva)

In [ ]:
from google.colab import files
import shutil

# Prompts mappa létrehozása
prompts_dir = "/content/prompts"
os.makedirs(prompts_dir, exist_ok=True)

print("📤 Töltsd fel a TXT fájljaidat:")
uploaded = files.upload()

# Fájlok áthelyezése
for filename in uploaded.keys():
    if filename.endswith('.txt'):
        shutil.move(filename, f"{prompts_dir}/{filename}")
        print(f"✓ {filename} feltöltve")

print("\n✓ Minden fájl feltöltve!")

## 3️⃣ Beállítások (Opcionális)

Módosítsd a generálási beállításokat:

In [ ]:
# ========== BEÁLLÍTÁSOK ==========

PERFORMANCE = "Extreme Speed"  # Speed, Quality, Extreme Speed
ASPECT_RATIO = "1344*704"      # 1344*704 (21:11), 1152*896, 896*1152, stb.
STYLES = ["SAI Comic Book"]    # Stílusok listája
MODEL = "juggernautXL_v8Rundiffusion.safetensors"  # Model neve
OUTPUT_FORMAT = "png"          # png vagy jpg
BATCH_SIZE = 32                # Max 32 kép egyszerre

# ========== PRESET MENTÉSE ==========

preset_path = "/content/Fooocus/presets/batch_auto.json"

custom_settings = {
    "default_performance": PERFORMANCE,
    "default_aspect_ratio": ASPECT_RATIO,
    "default_styles": STYLES,
    "default_model": MODEL,
    "default_output_format": OUTPUT_FORMAT
}

with open(preset_path, 'w') as f:
    json.dump(custom_settings, f, indent=4)

print("✓ Beállítások mentve:")
print(f"  Performance: {PERFORMANCE}")
print(f"  Felbontás: {ASPECT_RATIO}")
print(f"  Stílus: {STYLES}")
print(f"  Batch méret: {BATCH_SIZE}")

## 4️⃣ AUTOMATIKUS BATCH GENERÁLÁS

Ez a cella **automatikusan generálja** az összes képet!

In [ ]:
import sys
import time
from pathlib import Path

# Fooocus modulok betöltése
os.chdir("/content/Fooocus")
sys.path.insert(0, "/content/Fooocus")

# Inicializálás
print("🔧 Fooocus inicializálása...")
import modules.config as config
import modules.async_worker as worker
from modules.flags import Performance

# Beállítások alkalmazása
config.default_performance = Performance[PERFORMANCE.replace(" ", "_").upper()]
config.default_aspect_ratio = ASPECT_RATIO
config.default_styles = STYLES
config.default_output_format = OUTPUT_FORMAT

print("✓ Fooocus inicializálva\n")

# ========== PROMPTOK BEOLVASÁSA ==========

def read_prompts(file_path):
    """TXT fájl beolvasása."""
    prompts = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            line = line.strip()
            if line and not line.startswith('#'):
                prompts.append(line)
    return prompts

# TXT fájlok keresése
txt_files = sorted(Path(prompts_dir).glob("*.txt"))

if not txt_files:
    print("❌ Nincs TXT fájl a prompts mappában!")
else:
    print(f"📁 Talált fájlok: {len(txt_files)}\n")
    
    # ========== GENERÁLÁS ==========
    
    total_images = 0
    
    for txt_file in txt_files:
        print(f"{'='*60}")
        print(f"📝 Feldolgozás: {txt_file.name}")
        print(f"{'='*60}")
        
        prompts = read_prompts(txt_file)
        print(f"Promptok száma: {len(prompts)}\n")
        
        # Batch-enkénti feldolgozás
        for i in range(0, len(prompts), BATCH_SIZE):
            batch = prompts[i:i + BATCH_SIZE]
            batch_num = (i // BATCH_SIZE) + 1
            total_batches = (len(prompts) + BATCH_SIZE - 1) // BATCH_SIZE
            
            print(f"\n🎨 Batch {batch_num}/{total_batches} ({len(batch)} kép)")
            
            # Képek generálása egyesével
            for j, prompt in enumerate(batch, 1):
                print(f"  [{j}/{len(batch)}] {prompt[:60]}...")
                
                # Generálási parancs
                cmd = f'''python entry_with_update.py \
                    --preset batch_auto \
                    --always-high-vram \
                    --headless \
                    --prompt "{prompt}" \
                    --image-number 1'''
                
                # Futtatás
                os.system(cmd)
                total_images += 1
            
            print(f"  ✓ Batch {batch_num} kész!")
        
        print(f"\n✓ {txt_file.name} feldolgozva!\n")
    
    print(f"\n{'='*60}")
    print(f"🎉 GENERÁLÁS BEFEJEZVE!")
    print(f"{'='*60}")
    print(f"Összesen generált képek: {total_images}")

## 5️⃣ Generált Képek Letöltése

In [ ]:
import shutil
from pathlib import Path

# Outputs mappa tömörítése
print("📦 Képek tömörítése...")
output_dir = "/content/Fooocus/outputs"

if os.path.exists(output_dir):
    shutil.make_archive('/content/generated_images', 'zip', output_dir)
    
    # Statisztikák
    images = list(Path(output_dir).glob('**/*.png')) + list(Path(output_dir).glob('**/*.jpg'))
    total_size = sum(img.stat().st_size for img in images)
    
    print(f"\n📊 Statisztikák:")
    print(f"  Képek száma: {len(images)}")
    print(f"  Teljes méret: {total_size / 1024 / 1024:.2f} MB")
    
    # Letöltés
    print("\n⬇️ Letöltés indítása...")
    files.download('/content/generated_images.zip')
    
    print("\n✓ Képek letöltve! (generated_images.zip)")
else:
    print("❌ Nincs generált kép!")

---

## 💡 Tippek

- **Batch méret**: Csökkentsd 16-ra vagy 8-ra, ha memória problémák vannak
- **Performance**: `Extreme Speed` a leggyorsabb, `Quality` a legjobb minőség
- **Felbontás**: 1344x704 jó videókhoz, 1152x896 négyzetes képekhez

## 🐛 Hibaelhárítás

- **CUDA out of memory**: Csökkentsd a batch méretet
- **Lassú generálás**: Használj `Extreme Speed` performance-t
- **Nincs kép**: Ellenőrizd, hogy feltöltötted-e a TXT fájlokat

---

**GitHub**: [dwick90/Fooocus_txt_prompts](https://github.com/dwick90/Fooocus_txt_prompts)